In [4]:
#Add repo path to the system path
from pathlib import Path
import os, sys
repo_path= Path.cwd().resolve()
while '.gitignore' not in os.listdir(repo_path): # while not in the root of the repo
    repo_path = repo_path.parent #go up one level
sys.path.insert(0,str(repo_path)) if str(repo_path) not in sys.path else None

import pandas as pd


All inforamtion regarding the dataset is in the metadata and annotation csv files.<br>
For our purposes the annotations file contains all the info we need.

In [52]:
# paths
metadata_path = repo_path  /'data/vindr-mammo/metadata' / 'finding_annotations.csv'
# read
metadata = pd.read_csv(metadata_path, header=0)
# filter to images with no findings
metadata = metadata[metadata['finding_categories'] == "['No Finding']"]

# files found in the folder
directory_path = repo_path / 'data/vindr-mammo/images' / 'siemens15k_RGB'
# get only the file stem
file_names = [file.stem for file in directory_path.glob('*.png')]
# make dataframe with names
file_names = pd.DataFrame(file_names, columns=['image_id'])

# filter the metadata with the file names
metadata_actual = metadata.merge(file_names, on='image_id')
# add column called breast_density_prompt with diferent values depending the desity
metadata_actual['breast_density_prompt'] = metadata_actual['breast_density'].apply(lambda x: 'very low' if x == 'DENSITY A' else 'low' if x == 'DENSITY B' else 'high' if x == 'DENSITY C' else 'very high')
metadata_actual['prompt'] = 'a mammogram in ' + metadata_actual['view_position'] + ' view ' + 'with ' + metadata_actual['breast_density_prompt'] + ' density'
metadata_actual

,study_id,series_id,image_id,laterality,view_position,height,width,breast_birads,breast_density,finding_categories,finding_birads,xmin,ymin,xmax,ymax,split,breast_density_prompt,prompt
0,b8d273e8601f348d3664778dae0e7e0b,b36517b9cbbcfd286a7ae04f643af97a,d8125545210c08e1b1793a5af6458ee2,L,CC,3518,2800,BI-RADS 2,DENSITY C,['No Finding'],NaN,NaN,NaN,NaN,NaN,training,high,a mammogram in CC view with high density
1,b8d273e8601f348d3664778dae0e7e0b,b36517b9cbbcfd286a7ae04f643af97a,290c658f4e75a3f83ec78a847414297c,L,MLO,3518,2800,BI-RADS 2,DENSITY C,['No Finding'],NaN,NaN,NaN,NaN,NaN,training,high,a mammogram in MLO view with high density
2,b8d273e8601f348d3664778dae0e7e0b,b36517b9cbbcfd286a7ae04f643af97a,cd0fc7bc53ac632a11643ac4cc91002a,R,CC,3518,2800,BI-RADS 2,DENSITY C,['No Finding'],NaN,NaN,NaN,NaN,NaN,training,high,a mammogram in CC view with high density
3,b8d273e8601f348d3664778dae0e7e0b,b36517b9cbbcfd286a7ae04f643af97a,71638b1e853799f227492bfb08a01491,R,MLO,3518,2800,BI-RADS 2,DENSITY C,['No Finding'],NaN,NaN,NaN,NaN,NaN,training,high,a mammogram in MLO view with high density
4,8269f5971eaca3e5d3772d1796e6bd7a,d931832a0815df082c085b6e09d20aac,dd9ce3288c0773e006a294188aadba8e,L,CC,3518,2800,BI-RADS 1,DENSITY C,['No Finding'],NaN,NaN,NaN,NaN,NaN,training,high,a mammogram in CC view with high density
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13937,7fa05485d8e90a042cd57d5bb2206b57,fdca9e99351c85ba62a2a335a62f2a4e,fdeda828090d12552fb1c28e51f27507,R,MLO,3518,2800,BI-RADS 2,DENSITY C,['No Finding'],NaN,NaN,NaN,NaN,NaN,training,high,a mammogram in MLO view with high density
13938,a90e9bb390e07942ee8e749fb57003c3,f15b559d082a4fc6a9994e313745586f,5a5e4c9c66053045fefc0407076f0dea,L,CC,3518,2800,BI-RADS 1,DENSITY C,['No Finding'],NaN,NaN,NaN,NaN,NaN,training,high,a mammogram in CC view with high density
13939,a90e9bb390e07942ee8e749fb57003c3,f15b559d082a4fc6a9994e313745586f,052b8ed1c054b3a767ab304005933c55,L,MLO,3518,2800,BI-RADS 1,DENSITY C,['No Finding'],NaN,NaN,NaN,NaN,NaN,training,high,a mammogram in MLO view with high density
13940,a90e9bb390e07942ee8e749fb57003c3,f15b559d082a4fc6a9994e313745586f,b0d2fbc004346f845ed78209983f740d,R,CC,3518,2800,BI-RADS 2,DENSITY C,['No Finding'],NaN,NaN,NaN,NaN,NaN,training,high,a mammogram in CC view with high density


In [51]:
# drop all columns that are not prompt or image_id
metadata_prompt = metadata_actual[['image_id', 'prompt']]
# add exension to the end of the image_id
metadata_prompt['image_id'] = metadata_prompt['image_id'].astype(str) + '.png'
# change image_id to file_name
metadata_prompt = metadata_prompt.rename(columns={'image_id': 'file_name'})
# transform to json
metadata_prompt = metadata_prompt.to_json(orient='records', lines=True)
metadata_prompt

/tmp/ipykernel_1024410/905040472.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata_prompt['image_id'] = metadata_prompt['image_id'].astype(str) + '.png'


'{"file_name":"d8125545210c08e1b1793a5af6458ee2.png","prompt":"a mammogram in CC view with high density"}\n{"file_name":"290c658f4e75a3f83ec78a847414297c.png","prompt":"a mammogram in MLO view with high density"}\n{"file_name":"cd0fc7bc53ac632a11643ac4cc91002a.png","prompt":"a mammogram in CC view with high density"}\n{"file_name":"71638b1e853799f227492bfb08a01491.png","prompt":"a mammogram in MLO view with high density"}\n{"file_name":"dd9ce3288c0773e006a294188aadba8e.png","prompt":"a mammogram in CC view with high density"}\n{"file_name":"57fbdd278af5c8789a02b355c11620d4.png","prompt":"a mammogram in MLO view with high density"}\n{"file_name":"202d761a6b0f86faaeefc39ee18b1c53.png","prompt":"a mammogram in CC view with high density"}\n{"file_name":"acccc1727b61b261844d86aa9de53536.png","prompt":"a mammogram in MLO view with high density"}\n{"file_name":"a3d0e2394d7db36afab1b6e5e24da798.png","prompt":"a mammogram in CC view with high density"}\n{"file_name":"48b243704d16570155df12995a2